In [2]:
import numpy as np
from math import pi

In [3]:
X = np.linspace(-pi, pi, num=2000)
y = np.sin(X)

W = np.random.rand(4)

alpha = 1e-6

for step in range(2000):
    
    # polynomial 
    y_pred = W[0] + W[1] * X + W[2] * X ** 2 + W[3] * X ** 3

    loss = np.square(y_pred - y).sum()
    if step % 100 == 0:
        print(f'{step} loss: {loss}')

    loss_grad = 2 * (y_pred - y)

    grad = np.array([
        loss_grad.sum(),
        (X * loss_grad).sum(),
        (X ** 2 * loss_grad).sum() ,
        (X ** 3 * loss_grad).sum()
    ])

    W -= alpha * grad

0 loss: 265516.43417626707
100 loss: 583.7630371292918
200 loss: 389.168024081048
300 loss: 260.44373957339354
400 loss: 175.28984128106464
500 loss: 118.95708608270887
600 loss: 81.68954655168154
700 loss: 57.03399987724072
800 loss: 40.72175547114519
900 loss: 29.929082485577982
1000 loss: 22.78803946192646
1100 loss: 18.06291984172044
1200 loss: 14.93623746384328
1300 loss: 12.867164124422818
1400 loss: 11.497889326579992
1500 loss: 10.591677962266244
1600 loss: 9.991894891127654
1700 loss: 9.594898577634943
1800 loss: 9.332108966738467
1900 loss: 9.158144176454114


Okay, we can use numpy and its quite easy. But what if we will want to create something more complex.
The first uncofortable thing to notice is that we should calculate gradients manually. That is not cool. Let's try to change it by creating ou own classes.

In [4]:
import torch



x = torch.linspace(-pi, pi, 2000)
y = torch.sin(x)

power = torch.tensor([1,2,3])
polynomial = x.unsqueeze(-1).pow(power)

model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

loss_fn = torch.nn.MSELoss(reduction="sum")

for step in range(2000):
    y_pred = model(polynomial)

    loss = loss_fn(y_pred, y)

    model.zero_grad()

    loss.backward()

    with torch.no_grad():
        for param in model.parameters():
            param -= alpha * param.grad






In [5]:
import torch

dtype = torch.float
device = torch.device("cpu")


X = torch.linspace(-pi, pi, 2000, device=device, dtype=dtype)
y = torch.sin(X)

W = torch.randn((4,), device=device, dtype=dtype)

alpha = 1e-6

for step in range(2000):
    
    # polynomial 
    y_pred = W[0] + W[1] * X + W[2] * X ** 2 + W[3] * X ** 3

    loss = (y_pred - y).pow(2).sum().item()
    if step % 100 == 0:
        print(f'{step} loss: {loss}')

    loss_grad = 2 * (y_pred - y)

    grad = torch.tensor([
        loss_grad.sum(),
        (X * loss_grad).sum(),
        (X ** 2 * loss_grad).sum() ,
        (X ** 3 * loss_grad).sum()
    ])

    W -= alpha * grad

0 loss: 495570.0625
100 loss: 6180.21337890625
200 loss: 4090.698486328125
300 loss: 2708.70263671875
400 loss: 1794.640869140625
500 loss: 1190.0643310546875
600 loss: 790.1793212890625
700 loss: 525.6786499023438
800 loss: 350.72332763671875
900 loss: 234.99557495117188
1000 loss: 158.4435577392578
1100 loss: 107.80406951904297
1200 loss: 74.30517578125
1300 loss: 52.144309997558594
1400 loss: 37.48378372192383
1500 loss: 27.784563064575195
1600 loss: 21.367605209350586
1700 loss: 17.12200355529785
1800 loss: 14.312911987304688
1900 loss: 12.454136848449707


# Dataset
Ok, so we know what library would be used for learning students. 
But the actual question is to how make it interesting and fun. 
So let's make find some dataset.
* [Game of life](https://www.kaggle.com/competitions/conway-s-reverse-game-of-life)
* [Bicycle demand](https://www.kaggle.com/competitions/bike-sharing-demand) -- this one is actually cool, because the similar problem could be created for an RL lab and the difference between approaches would be clearly seen. 
* [Predicting an author](https://www.kaggle.com/competitions/painter-by-numbers) -- actually I wanted to do it. So the question is why not.


# Objectives

1, chcemy zrobić ten moduł w notatniku 
2, chcemy pokazać sieć neuronową jako kompozycję małych funkcji z których ona się składa (forward_pass, backward_pass, etc.)
3. chcemy pokazać podejście od dwóch stron: czyli najpierw jak trzeba się napracować, żeby zrobić coś za pomocą numpy, a potem jak łatwo da się to zrobić za pomocą pytorcha (albo innej podobnej biblioteki, która ma opcję liczenia gradientów)
4. Jako punkt startowy możemy użyć tego poradnika od pytorch - https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

--- od mnie 
5. Cause -> effect tego dlaczego API jest taki jaki jest i dlaczego sieci neuronowe działają


# What problems do I have with neural networks, because of how I was (wasn't) taught about NN

1. I don't know nothing about different types of pooling and it's benefits
2. I am not sure why we should initialize randomized weights
3. I don't know nothing about different types of activation functions and their benefits
4. I don't know how people actually compare different types of nets
5. I don't know nothing about different types of optimization algorithms 
6. I don't know nothing about computation graphs and why they are cool, and how they are built.
7. I am not sure about epochs and batches 


# Plan

I want to focus on the computation graph problem, because it appears in both pytorch and tensorflow. 

I will go from matrices and sinus function to explain the high level concept. 
After that, I am planning to go tep further and introduce the bicycle dataset. 
It would be harder to do with simple matrices, so instead we will need to create 
a more complicated PyTorch-like api. 

The last step would be to introduce a Pytorch and show that it is actually that simple.


# TODO

1. Learn about computational graph in pytorch and it benefits 
2. Try to implement it

In [6]:
from numpy.typing import NDArray
from typing import Callable
# 
class Weight:
    """
    Class representing single variable
    """
    def __init__(self, value: float, coefficient: Callable[[NDArray], NDArray] = lambda x: x) -> None:
        self.grad = 0
        self.coefficient = coefficient
        self.value = value

    def __call__(self, values: NDArray) -> NDArray:
        return self.value * self.coefficient(values)

    def compute_grad(self, X: NDArray, loss):
        self.grad = (self.coefficient(X) * loss).sum()

    def __str__(self) -> str:
        return f'{self.coefficient} * {self.value}'

    # backward will be needed for neural networs 


class Approximator:
    def __init__(self, weights: list[Weight]) -> None:
        self.weights = weights

    def __call__(self, values: NDArray) -> NDArray:
        a = np.sum([w(values) for w in self.weights], axis=0) # Summing up all weights in polynomial
        return a 
        

    def update_weights(self, update_step: float, X: NDArray, loss_grad: float):
        for w in self.weights:
            w.compute_grad(X, loss_grad)
            w.value -= update_step * w.grad 

    def __add__(self, weight: Weight):
        self.weights.append(weight)

    def __str__(self) -> str:
        return ' + '.join([str(w) for w in self.weights])


def sinus_polynom():
    return Approximator([
        Weight(np.random.rand()),
        Weight(np.random.rand(), lambda x: np.power(x, 1)),
        Weight(np.random.rand(), lambda x: np.power(x, 2)),
        Weight(np.random.rand(), lambda x: np.power(x, 3))
    ])
    


# ---------------------------------------

alpha = 1e-6
def train(approximator: Approximator, X: NDArray, Y: NDArray, n_steps: int):
    for s in range(n_steps):
        Y_approx = approximator(X)
        loss = np.square(Y - Y_approx).sum() 
        if s % 100 == 0:
            print(f'({s}) loss - {loss}')
        
        loss_grad = 2 * (Y_approx - Y)
        approximator.update_weights(alpha, X, loss_grad)

# ---------------------------------
X = np.linspace(-pi, pi, num=2000)
y = np.sin(X)
approximator = sinus_polynom()
train(approximator, X, y, 50000)
for i, x in enumerate(X[:50]):
    print(f'Test sin({i}) ({approximator})({i}): {np.sin(x)} {approximator(x)}')



(0) loss - 167209.68792349967
(100) loss - 17.830195949074362
(200) loss - 12.81887445316103
(300) loss - 10.59396094800275
(400) loss - 9.60607893415918
(500) loss - 9.167450254425438
(600) loss - 8.972695095419748
(700) loss - 8.886222017593761
(800) loss - 8.847827177493617
(900) loss - 8.830779516147715
(1000) loss - 8.823210198067304
(1100) loss - 8.819849351377545
(1200) loss - 8.818357104605507
(1300) loss - 8.817694533344689
(1400) loss - 8.817400345624598
(1500) loss - 8.817269723577184
(1600) loss - 8.817211726189438
(1700) loss - 8.81718597481705
(1800) loss - 8.817174540971395
(1900) loss - 8.817169464238997
(2000) loss - 8.817167210123223
(2100) loss - 8.81716620927514
(2200) loss - 8.817165764889403
(2300) loss - 8.817165567578057
(2400) loss - 8.81716547997001
(2500) loss - 8.817165441071232
(2600) loss - 8.817165423799818
(2700) loss - 8.81716541613115
(2800) loss - 8.817165412726192
(2900) loss - 8.817165411214361
(3000) loss - 8.817165410543094
(3100) loss - 8.8171654

Okay. 
It works, and I think you've got an idea, but you can easyli see, that approximations given by so simple polynomial are not very good. 
Let's try to make it better and create a Neural Network and make next iteration over our previous example! 


Basically neural network is a function that is a combination of layers. 
So, in order to create a network we should do next things: 
1. Implement logic of backward propagation
2. Implement dot product gradient (because weights are multiplied between layers)
3. Create layers of weights and after that connect them all (basically it would be an extension of an Approximator class)

# TODO - po spotkaniu

1. wizualizacja funkcji straty sinusu w zależności od wag. Ma to taką zaletę, że student zobaczy jaki wpływ mają wagi na loss: F(W) -> loss 
2. chihuahua i muffinki


# TODO

1. Make simple neural network based on `Approximators` to predict sinus value
2. Refactor code 

In [ ]:
from numpy.typing import NDArray
from typing import Callable

# In this step we will train without an activation function and we will see,
# that the training is very unefficient, because of very big changes in value.
# And in the next cell we will intorduce the cruicial part -- activation functions. 
# And basically it would show all nessesary components for having a Neural Network.

class MultivariateApproximator:
    def __init__(self, initial_weights: NDArray) -> None:
        """
        :param initial_weights: N-dimensional vector
        """
        self._weights = initial_weights

    def __call__(self, variables: NDArray) -> NDArray:
        """
        :param variables: M x N variables values
        :returns: M-dimensional vector

        """
        return self._weights @ variables

    def _update_weights(self, X: NDArray, loss: NDArray):
        """
        :param X: values for each loss was calculated
        :param loss: loss value for every input
        """
        """
        Basically, here I should create a Jacobian of Xs 
        multiplied by loss. 
        Am I right? 
        
        """


class NN:
    def __init__(self, layers: list[list[MultivariateApproximator]]) -> None:
        self.layers = layers

    def fit(self, X: NDArray, Y: NDArray, n_steps: int):
        for s in range(n_steps):
            Y_approx = self.predict(X)
            loss = np.square(Y - Y_approx).sum()
            if s % 100 == 0:
                print(f'({s}) loss - {loss}')
            
            loss_grad = 2 * (Y_approx - Y)
        
    def _backpropagate(self, X: NDArray, grad: NDArray):
        ...
        

    def predict(self, X: NDArray):
        ...
    


# ---------------------------------------

alpha = 1e-6
def train(approximator: Approximator, X: NDArray, Y: NDArray, n_steps: int):
    for s in range(n_steps):
        Y_approx = approximator(X)
        loss = np.square(Y - Y_approx).sum() 
        if s % 100 == 0:
            print(f'({s}) loss - {loss}')
        
        loss_grad = 2 * (Y_approx - Y)
        approximator.update_weights(alpha, X, loss_grad)

# ---------------------------------
X = np.linspace(-pi, pi, num=2000)
y = np.sin(X)
approximator = sinus_polynom()
train(approximator, X, y, 50000)
for i, x in enumerate(X[:50]):
    print(f'Test sin({i}) ({approximator})({i}): {np.sin(x)} {approximator(x)}')

